In [1]:
import os
import sqlite3

import nivapy3 as nivapy
import numpy as np
import pandas as pd
import utils

# Tiltaksovervakingen: opsjon for kvalitetskontroll av analysedata
## Notebook 1: Initial exploration and data cleaning

## 1. Background

Since 2012, the "Tiltaksovervakingen" project has collected data to assess the effects of liming at more than 200 stations across southern Norway. NIVALab was responsible for analysing water chemistry data for this project between 2012 and 2015, then VestfoldLAB took over from 2016 to August 2020, and from September 2020 onwards the analyses will be carried out by Eurofins. 

Data from 2012 to 2019 have been quality checked and are available in [Vannmiljø](https://vannmiljo.miljodirektoratet.no/) (`../data/vannmiljo_export_2012-19_2020-09-28.xlsx`). Data from VestfoldLAB from January to August 2020 are not yet quality assessed but have been provided by VestfoldLAB (`../data/vestfold_lab_data_to_2020-08-31.xls`). We have also agreed with Kjetil/Miljødirektoratet that future data from Eurofins will be provided using a similar Excel template (see e.g. e-mail from Kjetil received 24.09.2020 at 17.36).

**The aim of this workflow is to quality assess the "new" data and identify strange results ("outliers") for further investigation and reanalysis**. Since the project collects a lot of water samples, manually checking everything is not feasible. The task here is to implement semi-automatic data "screening" to identify a subset of the data for more thorough manual assessment. The first "new" dataset to be considered is the 2020 dataset from VestfoldLAB.

This notebook performs initial data exploration and cleaning, with the aim of creating a tidy dataset in SQLite that can be used for further analysis.

## 2. Create SQLite database to store results

Using a database will provide basic checks on data integrity and consistency. For this project, three tables will be sufficient:

 * Station locations and metadata
 * Parameters and units used by both VestfoldLAB and Vannmiljø, and conversion factors between these
 * Water chemistry data
 
The code below creates a basic database structure, which will be populated later.

In [2]:
# Create database
dbname = "kalk_data.db"
if os.path.exists(dbname):
    os.remove(dbname)
eng = sqlite3.connect(dbname, detect_types=sqlite3.PARSE_DECLTYPES)

# Turn off journal mode for performance
eng.execute("PRAGMA synchronous = OFF")
eng.execute("PRAGMA journal_mode = OFF")

# Create stations table
sql = (
    "CREATE TABLE stations "
    "( "
    "  fylke text NOT NULL, "
    "  vassdrag text NOT NULL, "
    "  station_name text NOT NULL, "
    "  station_number text, "
    "  vannmiljo_code text NOT NULL, "
    "  vannmiljo_name text, "
    "  utm_east real NOT NULL, "
    "  utm_north real NOT NULL, "
    "  utm_zone integer NOT NULL, "
    "  lon real NOT NULL, "
    "  lat real NOT NULL, "
    "  liming_status text NOT NULL, "
    "  comment text, "
    "  PRIMARY KEY (vannmiljo_code) "
    ")"
)
eng.execute(sql)

# Create parameters table
sql = (
    "CREATE TABLE parameters_units "
    "( "
    "  vannmiljo_name text NOT NULL UNIQUE, "
    "  vannmiljo_id text NOT NULL UNIQUE, "
    "  vannmiljo_unit text NOT NULL, "
    "  vestfold_lab_name text NOT NULL UNIQUE, "
    "  vestfold_lab_unit text NOT NULL, "
    "  vl_to_vm_conv_fac real NOT NULL, "
    "  min real NOT NULL, "
    "  max real NOT NULL, "
    "  PRIMARY KEY (vannmiljo_id) "
    ")"
)
eng.execute(sql)

# Create chemistry table
sql = (
    "CREATE TABLE water_chemistry "
    "( "
    "  vannmiljo_code text NOT NULL, "
    "  sample_date datetime NOT NULL, "
    "  lab text NOT NULL, "
    "  period text NOT NULL, "
    "  depth1 real, "
    "  depth2 real, "
    "  parameter text NOT NULL, "
    "  flag text, "
    "  value real NOT NULL, "
    "  unit text NOT NULL, "
    "  PRIMARY KEY (vannmiljo_code, sample_date, depth1, depth2, parameter), "
    "  CONSTRAINT vannmiljo_code_fkey FOREIGN KEY (vannmiljo_code) "
    "      REFERENCES stations (vannmiljo_code) "
    "      ON UPDATE NO ACTION ON DELETE NO ACTION, "
    "  CONSTRAINT parameter_fkey FOREIGN KEY (parameter) "
    "      REFERENCES parameters_units (vannmiljo_name) "
    "      ON UPDATE NO ACTION ON DELETE NO ACTION, "
    "  CONSTRAINT unit_fkey FOREIGN KEY (unit) "
    "      REFERENCES parameters_units (vannmiljo_unit) "
    "      ON UPDATE NO ACTION ON DELETE NO ACTION "
    ")"
)
eng.execute(sql)

## 3. Explore station data

Station details are stored in `../data/active_stations_2020.xlsx`, which is a tidied version of Øyvind's original file here:

    K:\Prosjekter\langtransporterte forurensninger\Kalk Tiltaksovervåking\12 KS vannkjemi\Vannlokaliteter koordinater_kun aktive stasj 2020.xlsx
    
Note that corrections (e.g. adjusted station co-ordinates) have been made to the tidied file, but not the original on `K:`. **The version in this repository should therefore been used as the "master" copy**.

In [3]:
# Read station data
stn_df = pd.read_excel(r"../data/active_stations_2020.xlsx", sheet_name="data")
stn_df = nivapy.spatial.utm_to_wgs84_dd(stn_df)

print("The following stations are missing spatial co-ordinates:")
stn_df.query("lat != lat")

The following stations are missing spatial co-ordinates:


,fylke,vassdrag,station_name,station_number,vannmiljo_code,vannmiljo_name,utm_east,utm_north,utm_zone,liming_status,comment,lat,lon


In [4]:
print("The following stations do not have a code in Vannmiljø:")
stn_df.query("vannmiljo_code != vannmiljo_code")

The following stations do not have a code in Vannmiljø:


,fylke,vassdrag,station_name,station_number,vannmiljo_code,vannmiljo_name,utm_east,utm_north,utm_zone,liming_status,comment,lat,lon


In [5]:
# Map
stn_map = nivapy.spatial.quickmap(
    stn_df.dropna(subset=["lat"]),
    lat_col="lat",
    lon_col="lon",
    popup="station_name",
    cluster=True,
    kartverket=True,
    aerial_imagery=True,
)

stn_map.save("../pages/stn_map.html")

stn_map

Based on the above:

 * Two stations are missing spatial reference information
 * 6 stations do not have codes in Vannmiljø
 
**Updated 09.10.2020:** Kjetil has supplied the missing station information (see e-mail received 08.10.2020 at 23.22). The code above reflects the new, complete dataset.

In [6]:
# Add to database
stn_df.dropna(subset=["vannmiljo_code", "lat"], inplace=True)
stn_df.to_sql(name="stations", con=eng, if_exists="append", index=False)

## 4. Parameters and units of interest

The file `../data/parameter_unit_mapping.xlsx` provides a lookup between parameter names & units used by VestfoldLAB and those in Vannmiljø. It also contains plausible ranges (using Vannmiljø units) for each parameter. These ranges have been chosen by using the values already in Vannmiljø as a reference. However, it looks as though some of the data in Vannmiljø might also be spurious, so it would be good to refine these ranges based on domain knowledge, if possible.

**To do:** Ask Øyvind to review and upadte the `max` and `min` ranges in the table below.

**Note:** Concentrations reported as *exactly* zero are likely to be errors, because most (all?) lab methods should report an LOQ instead.

In [7]:
# Read parameter mappings
par_df = utils.get_par_unit_mappings()

# Add to database
par_df.to_sql(name="parameters_units", con=eng, if_exists="append", index=False)

par_df

,vannmiljo_name,vannmiljo_id,vannmiljo_unit,vestfold_lab_name,vestfold_lab_unit,vl_to_vm_conv_fac,min,max
0,Temperatur,TEMP,°C,Temp,°C,1.000000,-10,30
1,pH,PH,<ubenevnt>,pH,enh,1.000000,1,10
2,Konduktivitet,KOND,mS/m,Kond,mS/m,1.000000,0,100
3,Total alkalitet,ALK,mmol/l,Alk,mmol/l,1.000000,0,2
4,Totalfosfor,P-TOT,µg/l P,Tot-P,µg/l,1.000000,0,500
5,Totalnitrogen,N-TOT,µg/l N,Tot-N,µg/l,1.000000,0,4000
6,Nitrat,N-NO3,µg/l N,NO3,µg/l,1.000000,0,2000
7,Totalt organisk karbon (TOC),TOC,mg/l C,TOC,mg/l,1.000000,0,100
8,Reaktivt aluminium,RAL,µg/l Al,RAl,µg/l,1.000000,0,500
9,Ikke-labilt aluminium,ILAL,µg/l Al,ILAl,µg/l,1.000000,0,500


## 5. Historic data from Vannmiljø

The Vannmiljø dataset is large and reading from Excel is slow; the code below takes a couple of minutes to run.

Note from the output below that **there are more than 1100 "duplicated" samples in the Vannmiljø dataset** i.e. where the station code, sample date, sample depth, lab and parameter name are all the same, but a different value is reported. It would be helpful to know why these duplicates were collected e.g. are these reanalysis values, where only one of the duplictaes should be used, or are they genuine (in which case should they be averaged or kept separate?). **For the moment, I will ignore these values**.

In [8]:
# Read historic data from Vannmiljø
his_df = utils.read_historic_data(r"../data/vannmiljo_export_2012-19_2020-11-04.xlsx")

# Tidy lab names for clarity
his_df["lab"].replace(
    {"NIVA": "NIVA (historic)", "VestfoldLAB AS": "VestfoldLAB (historic)"},
    inplace=True,
)

# Check duplicates
key_cols = [
    "vannmiljo_code",
    "sample_date",
    "lab",
    "depth1",
    "depth2",
    "par_unit",
]
his_dup_df = his_df[
    his_df.duplicated(
        key_cols,
        keep=False,
    )
].sort_values(key_cols)
his_dup_df.to_csv(r"../output/vannmiljo_duplicates.csv", index=False)

## Average duplicates
# his_df = his_df.groupby(key_cols).aggregate(
#    {
#        "flag": "first",
#        "value": "mean",
#    }
# ).reset_index()

# Remove all duplicates
his_df.drop_duplicates(subset=key_cols, keep=False, inplace=True)

# Add label for data period
his_df["period"] = "historic"

# Print summary
n_stns = len(his_df["vannmiljo_code"].unique())
print(f"The number of unique stations with data is: {n_stns}.\n")

print(
    f"There are {len(his_dup_df)} duplicated records (same station_code-date-depth-parameter, but different value).\n"
    "These will be averaged or ignored (depending on the settings above).\n"
)

his_df.head()

The number of unique stations with data is: 211.

There are 1140 duplicated records (same station_code-date-depth-parameter, but different value).
These will be averaged or ignored (depending on the settings above).



,vannmiljo_code,sample_date,lab,depth1,depth2,par_unit,flag,value,period
0,002-58798,2019-09-30,VestfoldLAB (historic),0.0,0.0,ALK_mmol/l,=,0.038,historic
1,002-58798,2019-12-02,VestfoldLAB (historic),0.0,0.0,ALK_mmol/l,=,0.044,historic
2,002-58798,2019-03-04,VestfoldLAB (historic),0.0,0.0,ALK_mmol/l,=,0.130,historic
3,002-58798,2019-01-07,VestfoldLAB (historic),0.0,0.0,ALK_mmol/l,=,0.058,historic
4,002-58798,2019-11-04,VestfoldLAB (historic),0.0,0.0,ALK_mmol/l,=,0.031,historic


## 6. New data from labs

The code below reads the Excel template provided by VestfoldLAB and reformats it to the same structure (parameter names, units etc.) as the data in Vannmiljø.

Again, the new dataset contains more than 100 "duplicates". It would be helpful to know more about these.

In [9]:
# Read new data
new_df = utils.read_data_template(
    r"../data/vestfold_lab_data_to_2020-08-31.xls", lab="VestfoldLAB (2020)"
)

# Check duplicates
new_dup_df = new_df[
    new_df.duplicated(
        key_cols,
        keep=False,
    )
].sort_values(key_cols)
new_dup_df.to_csv(r"../output/vestfoldlab_duplicates.csv", index=False)

## Average duplicates
# new_df = (
#    new_df.groupby(key_cols)
#    .aggregate(
#        {
#            "flag": "first",
#            "value": "mean",
#        }
#    )
#    .reset_index()
# )

# Remove all duplicates
new_df.drop_duplicates(subset=key_cols, keep=False, inplace=True)

# Add label for data period
new_df["period"] = "new"

print(
    f"There are {len(new_dup_df)} duplicated records (same station_code-date-depth-parameter, but different value).\n"
    "These will be averaged or ignored (depending on the settings above).\n"
)

new_df.head()

There are 116 duplicated records (same station_code-date-depth-parameter, but different value).
These will be averaged or ignored (depending on the settings above).



,vannmiljo_code,sample_date,lab,depth1,depth2,par_unit,flag,value,period
0,019-58793,2020-01-06,VestfoldLAB (2020),0.0,0.0,TEMP_°C,,3.4,new
1,019-58793,2020-02-03,VestfoldLAB (2020),0.0,0.0,TEMP_°C,,3.0,new
2,019-58793,2020-03-02,VestfoldLAB (2020),0.0,0.0,TEMP_°C,,3.5,new
3,019-58793,2020-04-06,VestfoldLAB (2020),0.0,0.0,TEMP_°C,,4.4,new
4,019-58793,2020-04-20,VestfoldLAB (2020),0.0,0.0,TEMP_°C,,6.4,new


## 7 . Combine

Combine the `historic` and `new` datasets into a single dataframe in "long" format.

In [10]:
# Combine
df = pd.concat([his_df, new_df], axis="rows")

# Separate par and unit
df[["parameter", "unit"]] = df["par_unit"].str.split("_", n=1, expand=True)
del df["par_unit"]

df.reset_index(drop=True, inplace=True)

df.head()

,vannmiljo_code,sample_date,lab,depth1,depth2,flag,value,period,parameter,unit
0,002-58798,2019-09-30,VestfoldLAB (historic),0.0,0.0,=,0.038,historic,ALK,mmol/l
1,002-58798,2019-12-02,VestfoldLAB (historic),0.0,0.0,=,0.044,historic,ALK,mmol/l
2,002-58798,2019-03-04,VestfoldLAB (historic),0.0,0.0,=,0.130,historic,ALK,mmol/l
3,002-58798,2019-01-07,VestfoldLAB (historic),0.0,0.0,=,0.058,historic,ALK,mmol/l
4,002-58798,2019-11-04,VestfoldLAB (historic),0.0,0.0,=,0.031,historic,ALK,mmol/l


## 8. Check data ranges

A simple method for preliminary quality control is to check whether parameter values are within sensible ranges (as defined in the `parameters_units` table; see Section 4 above). I believe this screening should be implemented differently for the `historic` (i.e. Vannmiljø) and `new` datsets, as follows:

 * For the `historic` data in Vannmiljø, values outside the plausible ranges should be **removed from the dataset entirely**. This is because we intend to use the Vannmiljø data as a reference against which new values will be compared, so it is important the dataset does not contain anything too strange. Ideally, the reference dataset should be carefully manually curated to ensure it is as good as possible, but I'm not sure we have the resouces in this project to thoroughly quality assess the data *already* in Vannmiljø. Dealing with any obvious issues is a good start, though
 
 * For the `new` data, values outside the plausible ranges should be highlighted and checked with the reporting lab
 
**Note:** At present, my code will remove any concentration values of exactly zero from the historic dataset. **Check with Øyvind whether this is too strict**.

In [11]:
# Check ranges
df = utils.check_data_ranges(df)


Checking data ranges for the 'historic' period.
    TEMP: Maximum value of 165.00 is greater than or equal to upper limit (30.00).
    KOND: Maximum value of 309.00 is greater than or equal to upper limit (100.00).
    LAL: Minimum value of 0.00 is less than or equal to lower limit (0.00).
    SO4: Minimum value of 0.00 is less than or equal to lower limit (0.00).
    CA: Minimum value of 0.00 is less than or equal to lower limit (0.00).

Checking data ranges for the 'new' period.
    P-TOT: Maximum value of 1200.00 is greater than or equal to upper limit (500.00).
    N-NO3: Maximum value of 2400.00 is greater than or equal to upper limit (2000.00).
    RAL: Minimum value of 0.00 is less than or equal to lower limit (0.00).
    ILAL: Minimum value of 0.00 is less than or equal to lower limit (0.00).
    LAL: Minimum value of 0.00 is less than or equal to lower limit (0.00).

Dropping problem rows from historic data.
    Dropping rows for TEMP.
    Dropping rows for KOND.
    Dropping

## 9. Summary

Points to note from the inital data exploration:

 * Two stations are missing spatial reference information and 6 do not yet have codes in Vannmiljø. Kjetil is aware of this and a complete list of stations plus metadata should be available soon (see e-mail received 29.09.2020 at 20.23). **Now fixed - see e-mail from Kjetil received 08.10.2020 at 23.22**
 
 * It would be useful if Øyvind could check and refine the `min` and `max` "plausible ranges" in my `parameters_units` table, since this provides the first layer of data screening. **Øyvind has suggested using `KOND > 100 mS/m` to remove samples contaminated by seawater**
 
 * I am not sure how to interpret values of *exactly* zero for concentrations (both in the "new" data and in the historic dataset from Vannmiljø). Are these values errors (i.e. the reporting is not correct), or is it common to ignore the LOQ and report zeros in some cases? 
 
  **Update 09.10.2019:** Some zero values from VestfoldLAB are caused by a problem with their Excel macro - see e-mail from Kjetil received 08.10.2020 at 13.37 for details. I have made the following changes to the data from VestfoldLAB:
  
    * **TOC**. Change values of 0 to 0.25 (cell comment "*Mindre enn verdi: <0.50*")
    * **TOC**. Despite the LOQ of 0.5, there are several TOC measurements with quantified values below this (e.g. 0.22, 0.28 etc.). **This is OK - see e-mail from Kjetil received 20.10.2020 at 12.12**
    * **CA**. Change values of 0 to 0.01 (cell comment "*Mindre enn verdi: <0.02*")
    * **SIO2**. Change values of 0 to 0.025 (cell comment "*Mindre enn verdi: <0.05*")
    * **RAl, LAl and ILAl**. Zeros are reported, but these are *not* flagged with an LOQ in the cell comments. Currently unchanged, but is this correct? **OK for now - see e-mail from Kjetil received 20.10.2020 at 12.12**
    
 
 * I am not sure how to interpret the "duplicated" samples (both in Vannmiljø and the "new" dataset"). By this, I am referring to samples where the station code, sample date, sample depth, lab and parameter name are all the same, but multiple values have been reported
 
  **Update 09.10.2019:** See e-mail sent to Kjetil 09.10.2020 at 15.23 for a more detailed overview of duplicates
 
 * Initial screening of the historic data from Vannmiljø suggests unusual values for `TEMP`, `LAL`, `SO4` and `CA`. These values are currently removed from the "reference" dataset, but the issues should perhaps be investigated further
 
 * Initial screening of the "new" data suggets unusual values for `P-TOT`, `N-NO3`, `TOC`, `RAL`, `ILAL`, `LAL`, `CA` and `SIO2`. These should be checked with VestfoldLAB
 
### Update 04.11.2020

We have agreed to:

 * Remove all duplicates from both "new" and "historic" datasets for the time being (done in Sections 5 and 6, above)
 * Correct the units for SiO2 from VestfoldLab already in Vannmiljø (the values were reported wrongly and are too small by a factor of ~468
 * Assume that nitrate ~ (nitrate + nitrite)

In [12]:
# Apply correction to historic SIO2
df["value"] = np.where(
    (df["lab"] == "VestfoldLAB (historic)") & (df["parameter"] == "SIO2"),
    df["value"] * 467.5432,
    df["value"],
)

# Reclassify (nitrate + nitrite) to nitrate
df["parameter"].replace({"N-SNOX": "N-NO3"}, inplace=True)

In [13]:
# Add to database
df.to_sql(
    name="water_chemistry",
    con=eng,
    if_exists="append",
    index=False,
    method="multi",
    chunksize=1000,
)

In [14]:
# Save historic reference data for Eurofins
eur_df = df.query("period == 'historic'")
eur_df.to_csv('../output/vannmiljo_reference_data_tidied.csv', index=False)